# 3_Concordances

- adding concordance information to the pelic_key_forms dataframe
- creating a concordance csv for manual annotation

#### Sections of the notebook
- [Initial setup](#Initial-setup)
- [Concordancer format](#Concordancer-format)
- [Concordancer contents](#Concordancer-contents)
- [Concordances dataframe](#Concordances-dataframe)
- [Concordances csv](#Concordances-csv)

In [1]:
# Importing necessary modules
import pandas as pd
import pprint
import pickle as pkl
import operator
import csv
from more_itertools import unique_everseen

# Setting preferred notebook format
%pprint # Turn off pretty printing
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # Shows all output, not just last item
pd.set_option('display.max_columns', 999) # Allow viewing of all columns

Pretty printing has been turned OFF


In [2]:
# Reading in pickle file
pelic_key_forms = pkl.load(open('pelic_key_forms.pkl', 'rb'))
pelic_key_forms.head()

,anon_id,text,level_id,native_language,gender,toks_nltk,toks_pos,lemmas,errors_in_text,len_errors_in_text,toks_corrected,lem_errors,len_lem_errors,lems_corrected,toks_pos_errors,len_toks_pos_errors,toks_pos_corrected,forms_in_text,len_forms
answer_id,,,,,,,,,,,,,,,,,,,
3,dk5,In my country we usually don't use tea bags. F...,4,Turkish,Female,"[In, my, country, we, usually, do, n't, use, t...","[(In, IN), (my, PRP$), (country, NN), (we, PRP...","[in, my, country, we, usually, do, n't, use, t...",[],0,"[In, my, country, we, usually, do, n't, use, t...",[],0,"[in, my, country, we, usually, do, n't, use, t...",[],0,"[(In, IN), (my, PR), (country, nn), (we, PR), ...","[(heat, nn)]",1
25,gc5,Last week I planned to go paintball match' but...,4,Turkish,Male,"[Last, week, I, planned, to, go, paintball, ma...","[(Last, JJ), (week, NN), (I, PRP), (planned, V...","[last, week, i, plan, to, go, paintball, match...",[],0,"[Last, week, I, planned, to, go, paintball, ma...",[],0,"[last, week, i, plan, to, go, paintball, match...",[],0,"[(last, jj), (week, nn), (I, PR), (planned, vv...","[(accepted, vv)]",1
30,er4,when you want to enjoy drinking a tea you have...,4,Arabic,Male,"[when, you, want, to, enjoy, drinking, a, tea,...","[(when, WRB), (you, PRP), (want, VBP), (to, TO...","[when, you, want, to, enjoy, drink, a, tea, yo...","[coract, finaly]",2,"[when, you, want, to, enjoy, drinking, a, tea,...","[coract, finaly]",2,"[when, you, want, to, enjoy, drink, a, tea, yo...","[(coract, JJ), (finaly, NN)]",2,"[(when, WR), (you, PR), (want, vv), (to, TO), ...","[(correct, jj)]",1
97,ea3,Here are a few instructions for typing in Engl...,4,Korean,Male,"[Here, are, a, few, instructions, for, typing,...","[(Here, RB), (are, VBP), (a, DT), (few, JJ), (...","[here, be, a, few, instruction, for, type, in,...",[],0,"[Here, are, a, few, instructions, for, typing,...",[],0,"[here, be, a, few, instruction, for, type, in,...",[],0,"[(here, rr), (are, vv), (a, DT), (few, jj), (i...","[(open, vv), (select, vv)]",2
111,cz7,Every one like a special kind of food. For me ...,4,Arabic,Male,"[Every, one, like, a, special, kind, of, food,...","[(Every, DT), (one, CD), (like, IN), (a, DT), ...","[every, one, like, a, special, kind, of, food,...",[],0,"[Every, one, like, a, special, kind, of, food,...",[],0,"[every, one, like, a, special, kind, of, food,...",[],0,"[(Every, DT), (one, CD), (like, IN), (a, DT), ...","[(special, jj), (specially, rr)]",2


### Concordancer format
Code in this section adapted from work (not public) by Dr Na-Rae Ha of the University of Pittsburgh, Department of Linguistics.

In [3]:
# Creating a function for finding a specific word in a tokenized text
def concord_toks(text, word, num):
    padding = [('')] * num             # non-word padding, in the specified size
    padded = padding + text + padding   # pad in front and back
    outlist = []
    for i in range(len(padded)):
        if padded[i] == word:
            outlist.append((padded[i-num:i], padded[i], padded[i+1:i+1+num]))
    return outlist

In [4]:
# Creating a function for flattening the resulting list from the previous function
def flatten(outlist):
    flatlist = []
    for x in outlist:
        flatlist.append((' '.join(x[0]), x[1], ' '.join(x[2])))
    return flatlist

In [5]:
# Creating a function to improve the appearance of the ouput from the previous list 
# to look like a typical concordance with node words aligned in a column.
def prettify(flatlist):
    return ['{:>40} {:^12} {:<40}'.format(x,y,z) for (x,y,z) in flatlist]

In [6]:
# Creating an overall function combing the three sub functions above
def concordance(text, word, num):
    #return prettify(flatten(concord_toks(text, word, num))) 
    ### Above line can be used to go directly to prettify version, but then difficult to sort concordance lines alphabetically
    return flatten(concord_toks(text, word, num))

### Concordancer contents
The concordance column is based on the forms in 'forms_in_text' column.

In [7]:
# Creating a function for getting the concordances based on a tokenized text and a list of the node words
def get_concs(tok_text, forms_list):
    conclist = []
    for x in tok_text:
        if x.lower() in [x[0] for x in forms_list]:
            conclist.append(concordance(tok_text, x, 5))
    return [x for y in list(unique_everseen(conclist)) for x in y]

In [8]:
# Applying the above function to create a new column with the concordance lines for each text
pelic_key_forms['concordance'] = pelic_key_forms[['toks_corrected', 'forms_in_text']]\
.apply(lambda x: get_concs(x[0],x[1]), axis=1)

In [9]:
def get_concs2(tok_text, forms_list):
    tok_text = [(x[0].lower(),x[1].lower()) for x in tok_text]
    conclist = []
    for x in tok_text:
        if x in forms_list:
            conclist.append(concordance([x[0] for x in tok_text], x[0], 5))
    return [x for y in list(unique_everseen(conclist)) for x in y]

In [10]:
tok_test = list(pelic_key_forms.loc[pelic_key_forms.index == 161].toks_pos_corrected)[0]
forms_list = list(pelic_key_forms.loc[pelic_key_forms.index == 161].forms_in_text)[0]

In [11]:
get_concs2(tok_test,forms_list)

[('    the', 'expectations', 'of women and men and'), ('role change occurs that peoples', 'expectations', 'of success begin to fall'), ('looking after the kids their', 'expectations', 'of having a delicious meal'), ('god created man and women', 'equal', 'to one another and to'), ('this teaches of respect and', 'equality', 'to everyone . a persons'), ('many problems with regard to', 'equality', 'amongst men and women and'), ('. the us system of', 'equality', 'amongst men and women is'), ('actually very much like the', 'equality', 'values of my islamic faith'), ('to work hard and is', 'expected', 'to succeed in whatever path'), ('differences , both genders are', 'expected', 'to succeed in their lives'), (', men and women are', 'expected', 'to succeed   ')]

#### Creating an 'offset' column to be able to refer back to this data later, i.e. their index in the text.

In [12]:
# Create function to return only items which match 'forms_in_text'
def get_offset(tok_list, forms):
    new_list =  [x.lower() for x in tok_list.copy()]
    new_forms = forms.copy()
    return [x for x in list(enumerate(new_list)) if x[1] in [x[0] for x in new_forms]]

# Creating an offset column by applying the above function
pelic_key_forms['offset'] = pelic_key_forms[['toks_corrected','forms_in_text']]\
.apply(lambda x: get_offset(x[0],x[1]), axis=1)

In [13]:
# Repeating the above process but with the POS information as well

# Need new function to return only items which match 'forms_in_text' including POS
def get_offset2(tok_pos_list, forms_pos):
    new_list =  tok_pos_list.copy()
    new_forms = forms_pos.copy()
    new_list = list(enumerate(new_list))
    return [x for x in new_list if x[1] in new_forms]

# Creating an offset_POS column by applying the above function
pelic_key_forms['offset_POS'] = pelic_key_forms[['toks_pos_corrected','forms_in_text']]\
.apply(lambda x: get_offset2(x[0],x[1]), axis=1)

In [14]:
pelic_key_forms.head(1)

,anon_id,text,level_id,native_language,gender,toks_nltk,toks_pos,lemmas,errors_in_text,len_errors_in_text,toks_corrected,lem_errors,len_lem_errors,lems_corrected,toks_pos_errors,len_toks_pos_errors,toks_pos_corrected,forms_in_text,len_forms,concordance,offset,offset_POS
answer_id,,,,,,,,,,,,,,,,,,,,,,
3,dk5,In my country we usually don't use tea bags. F...,4,Turkish,Female,"[In, my, country, we, usually, do, n't, use, t...","[(In, IN), (my, PRP$), (country, NN), (we, PRP...","[in, my, country, we, usually, do, n't, use, t...",[],0,"[In, my, country, we, usually, do, n't, use, t...",[],0,"[in, my, country, we, usually, do, n't, use, t...",[],0,"[(In, IN), (my, PR), (country, nn), (we, PR), ...","[(heat, nn)]",1,"[(like 20 minutes with low, heat, for it to be...","[(44, heat)]","[(44, (heat, nn))]"


In [15]:
# Doing a spot check that output is correct - this item has a capital letter (Nations) which was causing issues
pelic_key_forms.loc[pelic_key_forms.index==36464]
pelic_key_forms.loc[pelic_key_forms.index==36464].iloc[0,17] # forms in text is ok
pelic_key_forms.loc[pelic_key_forms.index==36464].iloc[0,19] # concordance is ok
pelic_key_forms.loc[pelic_key_forms.index==36464].iloc[0,20] # offset is ok
pelic_key_forms.loc[pelic_key_forms.index==36464].iloc[0,21] # offset_POS ok

,anon_id,text,level_id,native_language,gender,toks_nltk,toks_pos,lemmas,errors_in_text,len_errors_in_text,toks_corrected,lem_errors,len_lem_errors,lems_corrected,toks_pos_errors,len_toks_pos_errors,toks_pos_corrected,forms_in_text,len_forms,concordance,offset,offset_POS
answer_id,,,,,,,,,,,,,,,,,,,,,,
36464,ck0,The Earth's human population has grown steadi...,5,Korean,Male,"[The, Earth, 's, human, population, has, grown...","[(The, DT), (Earth, NNP), ('s, POS), (human, J...","[the, earth, 's, human, population, have, grow...",[continously],1,"[The, Earth, 's, human, population, has, grown...",[continously],1,"[the, earth, 's, human, population, have, grow...","[(continously, RB)]",1,"[(The, DT), (earth, nn), ('s, PO), (human, jj)...","[(nations, nn), (special, jj), (national, jj),...",4,"[(in 2009 by the United, Nations, . Some anthr...","[(25, nations), (160, special), (320, national...","[(25, (nations, nn)), (160, (special, jj)), (3..."


[('nations', 'nn'), ('special', 'jj'), ('national', 'jj'), ('continuously', 'rr')]

[('in 2009 by the United', 'Nations', '. Some anthropologists have cautioned'), ('have to only one except', 'special', "cases . China 's population"), ('CRUCIAL role in ENHANCING the', 'national', 'economy in the future .'), ('up children have been increasing', 'continuously', '. For instance , the')]

[(25, 'nations'), (160, 'special'), (320, 'national'), (481, 'continuously')]

[(25, ('nations', 'nn')), (160, ('special', 'jj')), (320, ('national', 'jj')), (481, ('continuously', 'rr'))]

In [16]:
# The offset columns need to be sorted by x[1] then x[0] (first the word then the index) 
# to match the concordance column.
for x in pelic_key_forms['offset']:
    x.sort(key = operator.itemgetter(1))
    
for x in pelic_key_forms['offset_POS']:
    x.sort(key = operator.itemgetter(1))
    
# And then also sorting the concordances alphabetically by node word
pelic_key_forms['concordance'] = [sorted(x, key=lambda x: x[1]) for x in pelic_key_forms.concordance]

### Concordances dataframe

In [17]:
# Creating a new dataframe based on pelic_key_forms but with each concordance as a row (rather than each text)
conc_df = pelic_key_forms[['anon_id','offset','offset_POS','concordance','level_id','native_language','gender','toks_pos_corrected']].reset_index()

In [18]:
# Creating a new column with tuples of the offset and the concordance (required later but easier to create here)
conc_df['offset_conc'] = list(zip(conc_df.offset_POS, conc_df.concordance))
conc_df['offset_conc'] = [list(zip(x[0],x[1])) for x in conc_df.offset_conc]

In [19]:
# Splitting this new column so that each item in the offset gets its own row (and doing some clean up)
conc_df = conc_df.offset_conc.apply(pd.Series) \
.merge(conc_df, left_index = True, right_index = True) \
.drop(['offset_conc'], axis = 1) \
.melt(id_vars = ['anon_id','concordance', 'offset', 'offset_POS','answer_id','level_id', 'native_language', 'gender','toks_pos_corrected'], value_name = 'offset_conc') \
.drop("variable", axis = 1) \
.dropna() \

In [20]:
# The 'offset_conc' column is the target - each row is a unique offset and concordance

# Dropping columns for now that contain duplicate info due to the above process.
conc_df = conc_df.drop(['offset', 'offset_POS','concordance'], axis = 1)

# Re-splitting the new offset_conc back into two columns
conc_df['offset_POS'] = [x[0] for x in conc_df.offset_conc]
conc_df['conc'] = [x[1] for x in conc_df.offset_conc] # NOTE: commas are around the node word (not from Ss)
conc_df.head()

,anon_id,answer_id,level_id,native_language,gender,toks_pos_corrected,offset_conc,offset_POS,conc
0,dk5,3,4,Turkish,Female,"[(In, IN), (my, PR), (country, nn), (we, PR), ...","((44, (heat, nn)), (like 20 minutes with low, ...","(44, (heat, nn))","(like 20 minutes with low, heat, for it to be ..."
1,gc5,25,4,Turkish,Male,"[(last, jj), (week, nn), (I, PR), (planned, vv...","((36, (accepted, vv)), (instead of me ? He, ac...","(36, (accepted, vv))","(instead of me ? He, accepted, my order and he..."
2,er4,30,4,Arabic,Male,"[(when, WR), (you, PR), (want, vv), (to, TO), ...","((13, (correct, jj)), (you have to know the, c...","(13, (correct, jj))","(you have to know the, correct, way to make it..."
3,ea3,97,4,Korean,Male,"[(here, rr), (are, vv), (a, DT), (few, jj), (i...","((100, (open, vv)), (space your writing homewo...","(100, (open, vv))","(space your writing homework ., Open, the Form..."
4,cz7,111,4,Arabic,Male,"[(Every, DT), (one, CD), (like, IN), (a, DT), ...","((4, (special, jj)), ( Every one like a, speci...","(4, (special, jj))","( Every one like a, special, kind of food . For)"


In [21]:
# Sorting dataframe by offset (word then offset number)
conc_df['offset_num'] = [x[0] for x in conc_df.offset_POS]
conc_df['offset_word_POS'] = [x[1] for x in conc_df.offset_POS]
conc_df = conc_df.sort_values(by = ['offset_word_POS', 'answer_id','offset_num']).reset_index(drop=True)

In [22]:
# Creating pickle of this dataframe for later use
conc_df.to_pickle("conc_df.pkl")

In [23]:
len(conc_df)

8746

In [24]:
# Dropping duplicates - from manual checking, it was found that some texts have two versions back to back, 
# some might be repetitions of task prompts, and some are mislabelled as version 1.
conc_df = conc_df.drop_duplicates(subset='conc', keep="first")
conc_df.head()
len(conc_df)

,anon_id,answer_id,level_id,native_language,gender,toks_pos_corrected,offset_conc,offset_POS,conc,offset_num,offset_word_POS
0,dy7,9638,5,Korean,Female,"[(Before, IN), (I, PR), (came, vv), (to, TO), ...","((520, (accept, IN)), (Most of meters in Pitts...","(520, (accept, IN))","(Most of meters in Pittsburgh, accept, only qu...",520,"(accept, IN)"
2,dy7,10662,5,Korean,Female,"[(Before, IN), (I, PR), (came, vv), (to, TO), ...","((620, (accept, IN)), (. Most meters in Pittsb...","(620, (accept, IN))","(. Most meters in Pittsburgh, accept, only qua...",620,"(accept, IN)"
3,ea9,26095,4,Arabic,Male,"[(What, WP), (the, DT), (reasons, nn), (of, IN...","((168, (accept, IN)), (in Japanese people are ...","(168, (accept, IN))","(in Japanese people are not, accept, in Saudi ...",168,"(accept, IN)"
4,gh9,39758,4,Japanese,Female,"[(argumentative, jj), (essay, nn), (:, :), (re...","((166, (accept, IN)), (responsibility by watch...","(166, (accept, IN))","(responsibility by watching their parents, acc...",166,"(accept, IN)"
6,fs9,47260,4,Arabic,Male,"[(everyone, nn), (have, vv), (many, jj), (goal...","((390, (accept, IN)), (the administration of m...","(390, (accept, IN))","(the administration of my department, accept, ...",390,"(accept, IN)"


7554

In [25]:
# Adding a new randomized index to reduce annotator fatigue
conc_df = conc_df.reset_index(drop = True)
conc_df = conc_df.reset_index(drop = False)
conc_df = conc_df.sample(frac=1)
conc_df = conc_df.reset_index(drop = True)

In [26]:
# Cleaning up the dataframe for easier viewing
conc_df.rename(columns= {'offset_POS':'offset', 'offset_word_POS':'node','offset_num':'node_index',
                         'native_language': 'L1', 'level_id':'level','toks_pos_corrected':'toks_pos',
                         'index':'orig_index'}, inplace=True)
conc_df = conc_df[['orig_index','anon_id','answer_id','level','L1','gender','node', 'node_index', 'offset', 
                   'conc', 'offset_conc','toks_pos']]

In [27]:
# Making all POS tags in toks_POS lower case - this is necessary in later workbooks
conc_df.toks_pos = conc_df.toks_pos.apply(lambda row:[(x[0],x[1].lower()) for x in row])

In [28]:
len(conc_df)
conc_df.head()

7554

,orig_index,anon_id,answer_id,level,L1,gender,node,node_index,offset,conc,offset_conc,toks_pos
0,7494,ey3,19175,4,Korean,Female,"(widen, vv)",390,"(390, (widen, vv))","(Finally , economic internet has, widen, their...","((390, (widen, vv)), (Finally , economic inter...","[(alvin, nn), (toffler, nn), (who, wp), (is, v..."
1,13,fg9,203,4,Arabic,Male,"(accept, vv)",62,"(62, (accept, vv))","(example , they did n't, accept, the checks fo...","((62, (accept, vv)), (example , they did n't, ...","[(I, pr), (was, vv), (working, vv), (as, in), ..."
2,4856,dq9,37104,5,Chinese,Female,"(nations, nn)",361,"(361, (nations, nn))","(the steel centers of the, nations, . About a ...","((361, (nations, nn)), (the steel centers of t...","[(658words, nn), (I, pr), (was, vv), (born, vv..."
3,2081,ay3,21959,3,Japanese,Female,"(continue, vv)",152,"(152, (continue, vv))","(enjoy good health when I, continue, eating it...","((152, (continue, vv)), (enjoy good health whe...","[(My, pr), (favorite, jj), (food, nn), (is, vv..."
4,4084,aq4,44324,4,Arabic,Female,"(heat, vv)",180,"(180, (heat, vv))","(it helps your body to, heat, when you get cut ,)","((180, (heat, vv)), (it helps your body to, he...","[(nutritional, jj), (benefits, nn), (of, in), ..."


In [29]:
# Checking the distribution of the different forms
forms_count = print(conc_df.node.value_counts().to_string())
forms_count

(special, jj)            775
(back, rr)               678
(continue, vv)           261
(structure, nn)          197
(national, jj)           192
(open, vv)               179
(accept, vv)             177
(open, jj)               151
(correct, jj)            142
(expect, vv)             140
(expected, vv)           129
(wide, jj)               129
(national, nn)           124
(select, vv)             122
(nation, nn)             120
(equal, jj)              116
(heat, nn)               115
(back, nn)               108
(exciting, jj)           102
(nations, nn)            100
(accepted, vv)            92
(opened, vv)              91
(confused, vv)            87
(correct, vv)             82
(expectations, nn)        74
(competitive, jj)         69
(correctly, rr)           65
(widely, rr)              64
(unexpected, jj)          59
(compete, vv)             57
(expectation, nn)         57
(construction, nn)        57
(excited, vv)             56
(opening, vv)             53
(excited, jj) 

In [30]:
# Total key family tokens at each level
conc_df.level.value_counts()

4    3551
5    3138
3     865
Name: level, dtype: int64

In [31]:
# Create pickle of this dataframe for later use
conc_df.to_pickle("conc_df.pkl")

### Concordances csv

In [32]:
# Zipping these columns together to prepare the concordance csv
conc_csv = zip(conc_df.answer_id, conc_df.offset, conc_df.conc)
conc_csv = list(conc_csv)

In [33]:
# Running the prettify function to prepare the csv for printing
conc_csv = [(x[0], x[1], prettify([x[2]])) for x in conc_csv]

In [34]:
%pprint
len(conc_csv)
conc_csv[:5]

Pretty printing has been turned ON


7554

[(19175,
  (390, ('widen', 'vv')),
  ['         Finally , economic internet has    widen     their range with services such          ']),
 (203,
  (62, ('accept', 'vv')),
  ["                  example , they did n't    accept    the checks for the soling               "]),
 (37104,
  (361, ('nations', 'nn')),
  ['                the steel centers of the   nations    . About a century ago                   ']),
 (21959,
  (152, ('continue', 'vv')),
  ['                enjoy good health when I   continue   eating it .                             ']),
 (44324,
  (180, ('heat', 'vv')),
  ['                   it helps your body to     heat     when you get cut ,                      '])]

In [35]:
# Write out conc_csv to csv
with open('conc_csv.csv', 'w', ) as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for answer_id, offset, conc in conc_csv:
        wr.writerow([answer_id, offset, conc])

135

135

137

138

134

...

134

136

136

134

137